In [ ]:
import pandas as pd
import numpy as np
import math as math

from google.colab import drive

In [ ]:
df = pd.read_stata("usa_00007.dta")
df

,year,perwt,sex,age,race,raced,wkswork2,incwage,qage,qsex,qrace,qwkswork2,qincwage,us2010a_schl,us2019a_schl
0,2010,169,male,46,white,white,50-52 weeks,52000,entered as written,entered as written,entered as written,original entry or inapplicable (not in universe),allocated,16,
1,2010,179,female,44,white,white,50-52 weeks,28500,entered as written,entered as written,entered as written,original entry or inapplicable (not in universe),allocated,16,
2,2010,59,male,44,black/african american/negro,black/african american/negro,50-52 weeks,49300,entered as written,entered as written,entered as written,original entry or inapplicable (not in universe),original entry or inapplicable (not in universe),16,
3,2010,74,male,44,white,white,50-52 weeks,50000,entered as written,entered as written,entered as written,original entry or inapplicable (not in universe),original entry or inapplicable (not in universe),16,
4,2010,87,female,41,white,white,50-52 weeks,15000,entered as written,entered as written,entered as written,original entry or inapplicable (not in universe),original entry or inapplicable (not in universe),16,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804533,2019,44,female,40,white,white,50-52 weeks,20000,entered as written,entered as written,entered as written,original entry or inapplicable (not in universe),original entry or inapplicable (not in universe),,20
804534,2019,68,female,41,white,white,n/a,0,entered as written,entered as written,entered as written,original entry or inapplicable (not in universe),original entry or inapplicable (not in universe),,19
804535,2019,63,male,41,white,white,50-52 weeks,118000,entered as written,entered as written,entered as written,original entry or inapplicable (not in universe),original entry or inapplicable (not in universe),,22
804536,2019,136,female,42,white,white,50-52 weeks,40000,entered as written,entered as written,entered as written,original entry or inapplicable (not in universe),original entry or inapplicable (not in universe),,22


In [ ]:
#Data Quality filtering 
df = df[df['qage']!='allocated']
df = df[df['qsex']!='allocated']
df = df[df['qrace']!='allocated']
df = df[df['qwkswork2']!='allocated']
df = df[df['qincwage']!='allocated']
df = df.drop(columns=['qage','qsex','qrace','qwkswork2','qincwage'])
df

,year,perwt,sex,age,race,raced,wkswork2,incwage,us2010a_schl,us2019a_schl
2,2010,59,male,44,black/african american/negro,black/african american/negro,50-52 weeks,49300,16,
3,2010,74,male,44,white,white,50-52 weeks,50000,16,
4,2010,87,female,41,white,white,50-52 weeks,15000,16,
5,2010,107,male,48,white,white,50-52 weeks,65000,21,
6,2010,86,female,43,white,white,50-52 weeks,61000,22,
...,...,...,...,...,...,...,...,...,...,...
804533,2019,44,female,40,white,white,50-52 weeks,20000,,20
804534,2019,68,female,41,white,white,n/a,0,,19
804535,2019,63,male,41,white,white,50-52 weeks,118000,,22
804536,2019,136,female,42,white,white,50-52 weeks,40000,,22


In [ ]:
#Complexity of the data has meant seperating hispanic from race to be technically challenging due to crossovers
#eg. (white and hispanic) 
df = df.drop(columns=["raced"])
df

,year,perwt,sex,age,race,wkswork2,incwage,us2010a_schl,us2019a_schl
2,2010,59,male,44,black/african american/negro,50-52 weeks,49300,16,
3,2010,74,male,44,white,50-52 weeks,50000,16,
4,2010,87,female,41,white,50-52 weeks,15000,16,
5,2010,107,male,48,white,50-52 weeks,65000,21,
6,2010,86,female,43,white,50-52 weeks,61000,22,
...,...,...,...,...,...,...,...,...,...
804533,2019,44,female,40,white,50-52 weeks,20000,,20
804534,2019,68,female,41,white,n/a,0,,19
804535,2019,63,male,41,white,50-52 weeks,118000,,22
804536,2019,136,female,42,white,50-52 weeks,40000,,22


In [ ]:
#Sub-sampling for black and white people only
df = df[(df['race']=='white')|(df['race']=='black/african american/negro')]
df['black'] = df['race'].apply(lambda x: 1 if (x=='black/african american/negro') else 0)
df = df.drop(columns=['race'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#Cleaning up the education variable
df["education"] = df["us2010a_schl"] + df["us2019a_schl"]
df = df.drop(columns = ["us2010a_schl","us2019a_schl"])
df['education']=df['education'].astype(str).astype(int)

In [ ]:
#Convert age to integer
df['age']=df['age'].astype(str).astype(int)

In [ ]:
#Convert year to integer
df['year']=df['year'].astype(str).astype(int)

In [ ]:
df

,year,perwt,sex,age,wkswork2,incwage,black,education
2,2010,59,male,44,50-52 weeks,49300,1,16
3,2010,74,male,44,50-52 weeks,50000,0,16
4,2010,87,female,41,50-52 weeks,15000,0,16
5,2010,107,male,48,50-52 weeks,65000,0,21
6,2010,86,female,43,50-52 weeks,61000,0,22
...,...,...,...,...,...,...,...,...
804533,2019,44,female,40,50-52 weeks,20000,0,20
804534,2019,68,female,41,n/a,0,0,19
804535,2019,63,male,41,50-52 weeks,118000,0,22
804536,2019,136,female,42,50-52 weeks,40000,0,22


In [ ]:
#Create a male dummy
df['male']=df['sex'].apply(lambda x: 1 if (x=="male") else 0)
df = df.drop(columns=['sex'])
df['male'].astype(str).astype(int)

2         1
3         1
4         0
5         1
6         0
         ..
804533    0
804534    0
804535    1
804536    0
804537    1
Name: male, Length: 579287, dtype: int64

In [ ]:
#Formatting number of weeks worked
df = df[df['wkswork2']!="n/a"]
df['wkswork2'] = df['wkswork2'].astype(str) 

#Transforming year wages into log weekly earnings
conditions = [
    (df['wkswork2'] == '1-13 weeks'),
    (df['wkswork2'] == '14-26 weeks'),
    (df['wkswork2'] == '27-39 weeks'),
    (df['wkswork2'] == '40-47 weeks'),
    (df['wkswork2'] == '48-49 weeks'),
    (df['wkswork2'] == '50-52 weeks')
    ]
choices = [7, 20, 33, 44, 48, 52]
#52 weeks because 50-51 is probably less likely
df['midpoint'] = np.select(conditions, choices)
df = df.drop(columns=["wkswork2"])

df = df[df['incwage']!=0]
df['lwage'] = np.log(df['incwage']/df['midpoint'])
df = df[df['lwage']>0]
df = df.drop(columns=['midpoint','incwage'])

In [ ]:
#Formatting education variables
df = df[df['education']>7]
conditions = [
    (df['education'] == 8) | (df['education'] == 9) | (df['education'] == 10) | (df['education'] == 11),
    (df['education'] == 12),
    (df['education'] == 13),
    (df['education'] == 14) | (df['education'] == 15),
    (df['education'] == 16) | (df['education'] == 17),
    (df['education'] == 18) | (df['education'] == 19),
    (df['education'] == 20), 
    (df['education'] == 21),
    (df['education'] == 22),
    (df['education'] == 23),
    (df['education'] == 24)
    ]
choices = [8,9,10,11,12,13,14,16,18,19,20]
df['educ'] = np.select(conditions, choices)
df = df.drop(columns=['education'])
#Note that in the original used by ACFV educ values can take on 14/15 and 16/17
#these represent different levels of associate and bachelors degrees respectively
#The data does not split these two levels of educations i.e. we have mapped
#14 = associates and 16 = bachelors, and no mapping for 15 and 17.

In [ ]:
#Generate experience variable
df['exper'] = df['age'] - df['educ'] - 6
df['expersq'] = df['exper']**2

In [ ]:
df['perwt'] = df['perwt']/df['perwt'].mean()

In [ ]:
df['male'].dtype

CategoricalDtype(categories=[1, 0], ordered=True)

In [ ]:
#we have some negative log wages due to some individuals have very low wages
#Does not seem like there are any imputed values

In [ ]:
df['black'].unique()

array([1, 0])

In [ ]:
#look at flags for data quality
#check inflation for wages

In [ ]:
df10 = df[df['year']==2010] # obs in 2010 sample
df19 = df[df['year']==2019] # obs in 2019 sample

In [ ]:
df10m = df[(df['year']==2010) & (df['male']==1)]
df10f = df[(df['year']==2010) & (df['male']==0)]
df19m = df[(df['year']==2019) & (df['male']==1)]
df19f = df[(df['year']==2019) & (df['male']==0)]

In [ ]:
df10m = df10m.drop(columns=['male'])
df10f = df10f.drop(columns=['male'])
df19m = df19m.drop(columns=['male'])
df19f = df19f.drop(columns=['male'])

In [ ]:
df10m.to_csv('/content/df10m')
df10f.to_csv('/content/df10f')
df19m.to_csv('/content/df19m')
df19f.to_csv('/content/df19f')